In [126]:
import re
import csv
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
#import nltk
from collections import Counter 
import matplotlib.pyplot as plt
from math import log

# On génère les série temporelle

In [10]:
def generate_first_type_time_series(D):
    return np.array([[np.sin(np.divide(2*np.pi*i*t, 64)) for t in range(128)] for i in range(1,D+1)])

In [12]:
def generate_second_type_time_series(D):
    fst_type = generate_first_type_time_series(D)
    result = []
    frac = 1/(D-1)
    for i in range(1,D+1):
        loc = []
        for t in range(1,129):
            fst = fst_type[i-1,t-1]
            snd=frac*np.sum(np.delete(fst_type[:,t-1], i-1, axis=0))
            loc.append(fst+snd)
        result.append(loc)
    return np.array(result)

In [169]:
def get_loader(D, MTS):
    data = []
    labels = []
    
    if MTS:
        series = generate_second_type_time_series(D)
    else : 
        series = generate_first_type_time_series(D)
     
    for serie in series :
        for i in range(0,64):
            data.append(serie[i:i+64])
            labels.append(serie[i+64])
    data = np.array(data)
    labels = np.array(labels) 
    train_data = TensorDataset(th.from_numpy(data).type(th.LongTensor), th.from_numpy(labels))
    train_loader = DataLoader(train_data, shuffle=False, batch_size=1)
    
    return train_loader

In [148]:
class GRU_forecaster(nn.Module):
    def __init__(self, nb_cells, hidden_size, input_size, rnn_dropout, window_size, bidirectional=False):
        super(GRU_forecaster, self).__init__()
        
        self.nb_cells = nb_cells
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.is_bidirectional = bidirectional
        
        self.gru = nn.GRU(input_size, hidden_size, nb_cells, batch_first = True, dropout=rnn_dropout,bidirectional=bidirectional)

        if bidirectional:    
            self.fc = nn.Linear(2*hidden_size, 1)
        else:
            self.fc = nn.Linear(hidden_size, 1)
        
        ###Par rapport au papier, k=nbr de filtres et m=hidden size
        self.convList = nn.ModuleList([nn.Conv1d(1,1,window_size-1) for i in range(32)])#32 fixed in the paper

        self.Wa = nn.Linear(hidden_size, 32)
        self.Wh = nn.Linear(hidden_size, hidden_size)
        self.Wv = nn.Linear(32, hidden_size)
        self.Whp = nn.Linear(hidden_size, 1)
        
        self.sigmoid = nn.Sigmoid()
        
        nn.init.xavier_uniform_(self.fc.weight.data)  # Xavier/Glorot init for tanh
        nn.init.zeros_(self.fc.bias.data)  # Xavier/Glorot init for tanh
        
            
    
    def forward(self, inputs, h0):
        inputs = inputs.view(1,-1,1).float()
        batch_size = inputs.size(0)
        rnn_out, h = self.gru(inputs, h0)
        hidden = rnn_out.transpose(1,2)[:,:,:-1]
        last_h = rnn_out[0,-1]
        hc = th.Tensor() #size m*k
        #l_out = self.getNbFeatures()
        
        for conv in self.convList:
            t = th.Tensor()
            for h in hidden[0]:
                h=h.view(1,1,-1)
                x = conv(h)
                t=th.cat([t,x],0)
            t = t.view(-1,1)
            hc=th.cat([hc,t],1)
        
        attention_weights = self.Wa(last_h).view(-1,1)
        attention_weights = (hc@attention_weights)
        attention_weights = self.sigmoid(attention_weights)
        
        
        vt = (attention_weights*hc).sum(dim=0) #ai * Hci
        
        
        hp = self.Wv(vt)+self.Wh(last_h)
        out = self.Whp(hp)
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        #hidden = th.Tensor(self.nb_cells, batch_size, self.hidden_size)
        if self.is_bidirectional:
            hidden = weight.new(2*self.nb_cells, batch_size, self.hidden_size).zero_()
        else:
            hidden = weight.new(self.nb_cells, batch_size, self.hidden_size).zero_()
        return hidden
    

In [170]:
nb_cells=1
learning_rate = 1e-2
hidden_size=150
input_size=1
gru=GRU_forecaster(nb_cells,hidden_size, input_size, 0.0,64)
loss_fn = nn.L1Loss()
D=6
loader = get_loader(D, False)

In [171]:
def train(model,lr, max_epochs=200):
    optim = th.optim.Adam(params=model.parameters(), lr=lr, weight_decay=1e-4)
    model.train()
    for i in range(max_epochs):
        n = 0 
        h = model.init_hidden(1)
        train_mean_loss=0
        for x, labels in loader:
            
            optim.zero_grad()
            h = h.data
            
            n+=1
            preds, _ = model(x, h)  
            
            preds=preds.view(1,-1)
            labels=labels.view(1,-1).float()
            loss = loss_fn(preds, labels)
            h = h.detach()
            train_mean_loss+=loss.item()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 5)
            optim.step()
        
            if(n%20==0):
                print("step : {}/{} ".format(n, 64*D))
                print("step loss : ", log(train_mean_loss/n,10))  
        
#         rnn_train_losses.append(train_mean_loss/len(train_data))
        
        print("EPOCH {}".format(i+1))
        print("Train Mean loss : ",log(train_mean_loss/n,10))
        print("----------------------------------------")

In [ ]:
train(gru, learning_rate)

step : 20/384 
step loss :  0.40367230204071036
step : 40/384 
step loss :  0.15602928429900448
step : 60/384 
step loss :  0.011942997414283327
step : 80/384 
step loss :  -0.08698546005476424
step : 100/384 
step loss :  -0.12861896671932857
step : 120/384 
step loss :  -0.15857198748649795
step : 140/384 
step loss :  -0.19866356670254323
step : 160/384 
step loss :  -0.20719121142180325
step : 180/384 
step loss :  -0.22311990324481182
step : 200/384 
step loss :  -0.23483106826459177
step : 220/384 
step loss :  -0.10811409213536774
step : 240/384 
step loss :  0.3918052067903433
step : 260/384 
step loss :  0.39986060619478525
